# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4935, uptime 0:00:16
mini_beamline                    RUNNING   pid 4936, uptime 0:00:16
random_walk                      RUNNING   pid 4937, uptime 0:00:16
random_walk_horiz                RUNNING   pid 4938, uptime 0:00:16
random_walk_vert                 RUNNING   pid 4939, uptime 0:00:16
simple                           RUNNING   pid 4940, uptime 0:00:16
thermo_sim                       RUNNING   pid 4941, uptime 0:00:16
trigger_with_pc                  RUNNING   pid 4942, uptime 0:00:16


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-15 18:45:38
Persistent Unique Scan ID: 'a9896a00-1dea-4eda-9110-71b342af8808'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 18:45:38.7 |     -1.000 |     102448 |


|         2 | 18:45:38.8 |     -0.500 |     104346 |
|         3 | 18:45:38.8 |      0.000 |     104136 |
|         4 | 18:45:38.8 |      0.500 |     103895 |
|         5 | 18:45:38.8 |      1.000 |     102120 |
+-----------+------------+------------+------------+
generator scan ['a9896a00'] (scan num: 1)



('a9896a00-1dea-4eda-9110-71b342af8808',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/a9896a00-1dea-4eda-9110-71b342af8808-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-15 18:45:39
Persistent Unique Scan ID: '3f18ee91-618d-44a0-a6fe-0a2ad2c34a7d'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 18:45:39.3 |          -1 | 1111486557 |


|         2 | 18:45:39.4 |          -0 | 1105872000 |


|         3 | 18:45:39.5 |           0 | 1094438601 |


|         4 | 18:45:39.6 |           0 | 1087560778 |


|         5 | 18:45:39.7 |           1 | 1081383482 |
+-----------+------------+-------------+------------+
generator scan ['3f18ee91'] (scan num: 2)





('3f18ee91-618d-44a0-a6fe-0a2ad2c34a7d',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-15 18:45:40
Persistent Unique Scan ID: 'e9f02ea0-dfcf-4574-8b47-6bd411aa98bc'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 18:45:40.3 | 1060973402 |


|         2 | 18:45:41.2 | 1073154774 |


|         3 | 18:45:42.2 | 1126790267 |


...

|        28 | 18:46:07.2 | 1108521259 |


|        29 | 18:46:08.2 | 1056546418 |


|        30 | 18:46:09.2 | 1071548592 |


+-----------+------------+------------+
generator count ['e9f02ea0'] (scan num: 3)





('e9f02ea0-dfcf-4574-8b47-6bd411aa98bc',)